In [ ]:
!pip install pyspark
!pip install -q findspark
!pip install pyspark-stubs
#!pip install pyfpgrowth
#!pip3 install mlxtend --user
#%pip install mlxtend --upgrade
# innstall java
#apt-get install openjdk-8-jdk-headless -qq > /dev/null 
#install spark (change the version number if needed)
#wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz 
#unzip the spark file to the current folder
#!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip install -q findspark
#!pip install sklearn

In [ ]:
import pandas as pd
import collections, numpy as np
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

In [ ]:
from pyspark.ml.fpm import FPGrowth
from pyspark.ml.fpm import PrefixSpan
# $example off$
from pyspark.sql import SparkSession,Row
from pyspark.sql.types import *
from pyspark.sql.functions import collect_set, col, count,explode,collect_list
#import pyspark.sql.functions as F

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
spark = SparkSession\
.builder\
.appName("FPGrowthExample")\
.getOrCreate()

In [ ]:
allergy = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/allergies.csv"
careplans = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/careplans.csv"
claims = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/claims.csv"
devices = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/devices.csv"
encounters = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/encounters.csv"
imaging_studies = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/imaging_studies.csv"
immunuzation = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/immunizations.csv"
medication = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/medications.csv"
observation = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/observations.csv"
organization = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/organizations.csv"
patinet = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/patients.csv"
payer_transaction = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/payer_transitions.csv"
payers = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/payers.csv"
procedure = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/procedures.csv"
providers = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/providers.csv"
supplies = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/supplies.csv"
claim_transaction = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/claims_transactions.csv"
condition = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/conditions.csv"

In [ ]:
allergy = pd.read_csv(allergy)
careplans = pd.read_csv(careplans)
claims = pd.read_csv(claims)
devices = pd.read_csv(devices)
encounters = pd.read_csv(encounters)
imaging_studies = pd.read_csv(imaging_studies)
immunuzation = pd.read_csv(immunuzation)
medication = pd.read_csv(medication)
observation = pd.read_csv(observation)
organization = pd.read_csv(organization)
patient = pd.read_csv(patinet)
payer_transaction = pd.read_csv(payer_transaction)
payers = pd.read_csv(payers)
procedure = pd.read_csv(procedure)
providers = pd.read_csv(providers)
supplies = pd.read_csv(supplies)
claim_transaction = pd.read_csv(claim_transaction)
condition = pd.read_csv(condition)

In [ ]:
allergy.head()
print(encounters.head())
#patinet = patinet.select("*").toPandas()
claims.head()

In [ ]:
alg_pat = pd.merge(allergy, patient, left_on="PATIENT", right_on="Id")
print(alg_pat.columns)
print(len(alg_pat['Id'].unique())," | ",alg_pat.shape)

In [ ]:
enco_alg_pat = pd.merge(encounters, alg_pat, left_on="PATIENT", right_on="Id")
print(enco_alg_pat.columns)
print(len(enco_alg_pat['Id_y'].unique())," | ",enco_alg_pat.shape)

In [ ]:
enco_alg_pat_prov = pd.merge(enco_alg_pat,providers, left_on="PROVIDER", right_on="Id")
enco_alg_pat_prov.columns

In [ ]:
print(len(enco_alg_pat_prov['Id_y'].unique())," | ",enco_alg_pat_prov.shape)

In [ ]:
enco_alg_pat_prov_carpln = pd.merge(enco_alg_pat_prov,careplans, left_on="Id_y",right_on="PATIENT")
print(enco_alg_pat_prov_carpln.columns)

In [ ]:
df_columns = enco_alg_pat_prov_carpln.columns.values.tolist()
print(df_columns)
column_dup = [k for k,v in Counter(df_columns).items() if v > 1]
print(column_dup)

In [ ]:
enco_alg_pat_prov_carpln.set_axis(['Id_w', 'START_x', 'STOP_x', 'PATIENT_x', 'ORGANIZATION_x', 'PROVIDER',
                                   'PAYER', 'ENCOUNTERCLASS', 'CODE_x', 'DESCRIPTION_x',
                                   'BASE_ENCOUNTER_COST', 'TOTAL_CLAIM_COST', 'PAYER_COVERAGE',
                                   'REASONCODE_x', 'REASONDESCRIPTION_x', 'START_y', 'STOP_y', 'PATIENT_y',
                                   'ENCOUNTER_x', 'CODE_y', 'SYSTEM', 'DESCRIPTION_y', 'TYPE', 'CATEGORY',
                                   'REACTION1', 'DESCRIPTION1', 'SEVERITY1', 'REACTION2', 'DESCRIPTION2',
                                   'SEVERITY2', 'Id_y', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS',
                                   'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL',
                                   'RACE', 'ETHNICITY', 'GENDER_x', 'BIRTHPLACE', 'ADDRESS_x', 'CITY_x',
                                   'STATE_x', 'COUNTY', 'ZIP_x', 'LAT_x', 'LON_x', 'HEALTHCARE_EXPENSES',
                                   'HEALTHCARE_COVERAGE', 'Id_x', 'ORGANIZATION_y', 'NAME', 'GENDER_y',
                                   'SPECIALITY', 'ADDRESS_y', 'CITY_y', 'STATE_y', 'ZIP_y', 'LAT_y',
                                   'LON_y', 'UTILIZATION', 'Id_z', 'START', 'STOP', 'PATIENT',
                                   'ENCOUNTER_y', 'CODE', 'DESCRIPTION', 'REASONCODE_y','REASONDESCRIPTION_y'], axis=1, inplace=True)
enco_alg_pat_prov_carpln.columns

In [ ]:
print(enco_alg_pat_prov_carpln.columns)

In [ ]:
print(len(enco_alg_pat_prov_carpln['Id_y'].unique())," | ",enco_alg_pat_prov_carpln.shape)

In [ ]:
enco_alg_pat_prov_carpln_claim = pd.merge(enco_alg_pat_prov_carpln,claims, left_on="Id_y",right_on="PATIENTID")
print(enco_alg_pat_prov_carpln_claim.columns)

In [ ]:
df_columns = enco_alg_pat_prov_carpln.columns.values.tolist()
print(df_columns)
column_dup = [k for k,v in Counter(df_columns).items() if v > 1]
print(column_dup)

In [ ]:
from pyspark.ml.fpm import FPGrowth
from pyspark.ml.fpm import PrefixSpan
# $example off$
from pyspark.sql import SparkSession,Row
from pyspark.sql.types import *
from pyspark.sql.functions import collect_set, col, count,explode,collect_list
#import pyspark.sql.functions as F

In [ ]:
spark = SparkSession\
.builder\
.appName("FPGrowthExample")\
.getOrCreate()

In [ ]:
allergy = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/allergies.csv"
careplans = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/careplans.csv"
claims = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/claims.csv"
devices = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/devices.csv"
encounters = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/encounters.csv"
imaging_studies = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/imaging_studies.csv"
immunuzation = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/immunizations.csv"
medication = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/medications.csv"
observation = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/observations.csv"
organization = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/organizations.csv"
patinet = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/patients.csv"
payer_transaction = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/payer_transitions.csv"
payers = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/payers.csv"
procedure = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/procedures.csv"
providers = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/providers.csv"
supplies = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/supplies.csv"
claim_transaction = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/claims_transactions.csv"
condition = "/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/conditions.csv"

In [ ]:
allerg = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(allergy))
careplan = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(careplans))
claim = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(claims))
device = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(devices))
encounter = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(encounters))
imaging_studie = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(imaging_studies))
immunuzatio = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(immunuzation))
medicatio = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(medication))
observatio = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(observation))
organizatio = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(organization))
patien = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(patinet))
payer_transactio = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(payer_transaction))
payer = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(payers))
procedur = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(procedure))
provider = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(providers))
supplie = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(supplies))
claim_transactio = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(claim_transaction))
conditio = (spark.read.format("csv").options(header="true").option("inferSchema","true").load(condition))

In [ ]:
allerg.createOrReplaceTempView("aller")
careplan.createOrReplaceTempView("carepla")
claim.createOrReplaceTempView("clai")
device.createOrReplaceTempView("devic")
encounter.createOrReplaceTempView("encounte")
imaging_studie.createOrReplaceTempView("imging_studi")
immunuzatio.createOrReplaceTempView("immunizati")
medicatio.createOrReplaceTempView("medicati")
observatio.createOrReplaceTempView("observati")
organizatio.createOrReplaceTempView("organizati")
patien.createOrReplaceTempView("patie")
payer_transactio.createOrReplaceTempView("payer_transacti")
payer.createOrReplaceTempView("paye")
procedur.createOrReplaceTempView("procedu")
provider.createOrReplaceTempView("provide")
supplie.createOrReplaceTempView("suppli")
claim_transactio.createOrReplaceTempView("claim_transacti")
conditio.createOrReplaceTempView("conditi")

In [ ]:
pat_data1 = spark.sql("select patie.Id, encounte.ENCOUNTERCLASS,encounte.TOTAL_CLAIM_COST from encounte, patie where patie.Id == encounte.PATIENT")
#print(pat_data)
print(pat_data1.schema.names)
print()
print(pat_data1.show())

In [ ]:
pat1 = (spark.read.format("csv").options(header="true").option("inferSchema","true").load("/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/patients.csv"))
#pat1 = spark.read.csv("/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/patients.csv", header=True, inferSchema=True)
pat1.createOrReplaceTempView("patient")
pat1.printSchema()
#pat1.columns


In [ ]:
enco1 = (spark.read.format("csv").options(header="true").option("inferSchema","true").load("/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/encounters.csv"))
#pat1 = spark.read.csv("/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/patients.csv", header=True, inferSchema=True)
enco1.createOrReplaceTempView("encounter")
print(enco1.printSchema())
print(enco1.show())
#pat1.columns

In [ ]:
#enco_pat = pd.merge(enco, pat, left_on="PATIENT", right_on="Id")
enco1_pat1 = enco1.join(pat1,enco1.PATIENT ==  pat1.Id, "inner")
enco1_pat1.show(30)
enco1_pat1.count()

In [ ]:
#enco1 = (spark.read.format("csv").options(header="true").load(""))
#enco1 = spark.read.csv("/content/drive/MyDrive/Deep Learning/Frequent Pattern Mining/encounters.csv", header=True, inferSchema=True)
#enco1 = spark.createDataFrame(enco1)
#enco1 = enco1.select(explode("COUNTERCLASS").alias("c"))
#print(enco1.createOrReplaceTempView("encounter"))
#enco1.columns


In [ ]:

#rawData = spark.sql("select p.product_name, o.order_id from products p inner join order_products_train o where o.product_id = p.product_id")

pat_data = spark.sql("select p.Id, e.ENCOUNTERCLASS,e.TOTAL_CLAIM_COST from encounter e , patient p where p.Id == e.PATIENT")
#print(pat_data)
print(pat_data.schema.names)
#print(pat_data.printSchema())
#pat_class_schema = pat_data.groupBy('Id').agg(collect_set('ENCOUNTERCLASS').alias('items'))
print(pat_data.show())
pat_data.count()

In [ ]:
pat1_class1 = pat_data.groupBy('Id').agg(collect_list("TOTAL_CLAIM_COST").alias('items1'),collect_list("ENCOUNTERCLASS").alias('items2'))
pat1_class1.createOrReplaceTempView('class1')
print(pat1_class1.count())
pat1_class1.show()

In [ ]:
pat_dataDF = pat_data.select("*").toPandas()
print(pat_dataDF['TOTAL_CLAIM_COST'].isna().sum() )
class1DF = pat1_class1.select("*").toPandas()
print(class1DF['items1'].isna().sum() )

In [ ]:
pat1_class2 = pat_data.groupBy('Id').agg(collect_set('ENCOUNTERCLASS').alias('items'))
pat1_class2.createOrReplaceTempView('class2')
print(pat1_class2.count())
pat1_class2.show()

In [ ]:
pat1_class3 = pat_data.groupBy('Id').agg(collect_set('TOTAL_CLAIM_COST').alias('items'))
pat1_class3.createOrReplaceTempView('class3')
print(pat1_class3.count())
pat1_class3.show()

In [ ]:
#%scala
%time
from pyspark.ml.fpm import FPGrowth

# Extract out the items 
class1_itemsDF = spark.sql("select items from class3")#.as[Array[String]].toDF("items")

################# IT IS SHOWING PREDICTED DATA OF TOTAL_CLAIM_COST

# Use FPGrowth
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.006, minConfidence=0.006)
model = fpGrowth.fit(class1_itemsDF)
# Display frequent itemsets.
model.freqItemsets.show()
mostPopularPatClass = model.freqItemsets

model.associationRules
rules_TClaim_cost = model.associationRules
rules_TClaim_cost.show()  #giving ERROR
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(class1_itemsDF).show()
rules_TClaim_cost.createOrReplaceTempView("rules_TClaim_cost_table")

In [ ]:
#%scala
%time
from pyspark.ml.fpm import FPGrowth

####################### IT IS SHOWING PREDICTED DATA OF ENCOUNTERCLASS

# Extract out the items 
class1_itemsDF = spark.sql("select items from class2")#.as[Array[String]].toDF("items")

# Use FPGrowth
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.006, minConfidence=0.006)
model = fpGrowth.fit(class1_itemsDF)
# Display frequent itemsets.
model.freqItemsets.show()
mostPopularPatClass = model.freqItemsets

model.associationRules
rules_data = model.associationRules
rules_data.show()  #giving ERROR
# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(class1_itemsDF)
#rule.createOrReplaceTempView("rule_table")

In [ ]:
rulesDF = rules_data.select("*").toPandas()
rulesDF.head()

In [ ]:
rulesDF[rulesDF["antecedent"].apply(lambda x: "urgentcare" in str(x))].sort_values(ascending=False,by='lift')

In [ ]:
rulesTC_cost = rules_TClaim_cost.select("*").toPandas()
rulesTC_cost.head()

In [ ]:
rulesTC_cost[rulesTC_cost["antecedent"].apply(lambda x: "urgentcare" in str(x))].sort_values(ascending=False,by='lift')